In [1]:
#part 1: prepares data for a DNDS run.
#part 2: after running DNDS prepares data for plotting 
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re
import math
from collections import Counter

sys.path.append('/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts')
import configuration_util
import analysis_utils
import get_gene_and_cohort_list_utils

filePathDict = configuration_util.get_all_files_path_dict()

In [41]:
allImpactMutsUnfiltered = pd.read_csv(filePathDict['IMPACT_BASE_MAF_WITH_SYNONYMOUS'])


In [19]:
cancerTypesToRunDNDSOn = set(['Colorectal Cancer', 'Endometrial Cancer', 'Glioma',
                              'Prostate Cancer', 'Esophagogastric Cancer', 'Bladder Cancer'])

hypermutantIdDir= pathPrefix +'/juno/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds'
idsDict = {}
for cancerType in cancerTypesToRunDNDSOn:
    fileName = re.sub(' ', '_', cancerType)
    for status in ['Normal', 'Hypermutated']:
        fileNameAdj = fileName + '_' + status
        ids = get_gene_and_cohort_list_utils.get_ids_by_hypermutant_status(hypermutantIdDir=hypermutantIdDir, cancerType=cancerType, hypermutantStatus = status)
        idsDict[fileNameAdj] = ids
        

In [27]:
writeDir = pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/mafsForDndsCV/'
for cohort, ids in idsDict.items():
    mutations = allImpactMutsUnfiltered[allImpactMutsUnfiltered['Tumor_Sample_Barcode'].isin(ids)]
    #FIX THE COLUMN NAMES
    renameDict = {'Tumor_Sample_Barcode': 'sampleID', 'Chromosome': 'chr', 'Start_Position': 'pos',
                'Reference_Allele': 'ref', 'Tumor_Seq_Allele2': 'alt', 'Hugo_Symbol': 'gene', 'hgvs': 'HGVSp_Short' 
                }
    mutations = mutations.rename(columns=renameDict)
    mutations = mutations[renameDict.values()]
    filename = writeDir + cohort + '_DNDS_maf.tsv'
    print 'writing', filename
    mutations.to_csv(filename, index=False, sep='\t')
    

writing /Users/friedman/Desktop/mnt/juno/work/taylorlab/friedman/myAdjustedDataFiles/mafsForDndsCV/Glioma_Normal_DNDS_maf.tsv
writing /Users/friedman/Desktop/mnt/juno/work/taylorlab/friedman/myAdjustedDataFiles/mafsForDndsCV/Glioma_Hypermutated_DNDS_maf.tsv
writing /Users/friedman/Desktop/mnt/juno/work/taylorlab/friedman/myAdjustedDataFiles/mafsForDndsCV/Esophagogastric_Cancer_Hypermutated_DNDS_maf.tsv
writing /Users/friedman/Desktop/mnt/juno/work/taylorlab/friedman/myAdjustedDataFiles/mafsForDndsCV/Colorectal_Cancer_Hypermutated_DNDS_maf.tsv
writing /Users/friedman/Desktop/mnt/juno/work/taylorlab/friedman/myAdjustedDataFiles/mafsForDndsCV/Esophagogastric_Cancer_Normal_DNDS_maf.tsv
writing /Users/friedman/Desktop/mnt/juno/work/taylorlab/friedman/myAdjustedDataFiles/mafsForDndsCV/Bladder_Cancer_Hypermutated_DNDS_maf.tsv
writing /Users/friedman/Desktop/mnt/juno/work/taylorlab/friedman/myAdjustedDataFiles/mafsForDndsCV/Bladder_Cancer_Normal_DNDS_maf.tsv
writing /Users/friedman/Desktop/mnt

In [28]:
nucleosomeInfo = pd.read_table(pathPrefix + '/ifs/res/taylorlab/ang46/ext/resources/nucleosome_dyads_genic_hg19.bed')

In [42]:
impact468genes = set(['ABL1', 'ACVR1', 'AGO2', 'AKT1', 'AKT2', 'AKT3', 'ALK', 'ALOX12B', 'AMER1', 'ANKRD11', 'APC', 'AR', 'ARAF', 'ARID1A', 'ARID1B', 'ARID2', 'ARID5B', 'ASXL1', 'ASXL2', 'ATM', 'ATR', 'ATRX', 'AURKA', 'AURKB', 'AXIN1', 'AXIN2', 'AXL', 'B2M', 'BABAM1', 'BAP1', 'BARD1', 'BBC3', 'BCL10', 'BCL2', 'BCL2L1', 'BCL2L11', 'BCL6', 'BCOR', 'BIRC3', 'BLM', 'BMPR1A', 'BRAF', 'BRCA1', 'BRCA2', 'BRD4', 'BRIP1', 'BTK', 'CALR', 'CARD11', 'CARM1', 'CASP8', 'CBFB', 'CBL', 'CCND1', 'CCND2', 'CCND3', 'CCNE1', 'CD274', 'CD276', 'CD79A', 'CD79B', 'CDC42', 'CDC73', 'CDH1', 'CDK12', 'CDK4', 'CDK6', 'CDK8', 'CDKN1A', 'CDKN1B', 'CDKN2A', 'CDKN2B', 'CDKN2C', 'CEBPA', 'CENPA', 'CHEK1', 'CHEK2', 'CIC', 'CREBBP', 'CRKL', 'CRLF2', 'CSDE1', 'CSF1R', 'CSF3R', 'CTCF', 'CTLA4', 'CTNNB1', 'CUL3', 'CXCR4', 'CYLD', 'CYSLTR2', 'DAXX', 'DCUN1D1', 'DDR2', 'DICER1', 'DIS3', 'DNAJB1', 'DNMT1', 'DNMT3A', 'DNMT3B', 'DOT1L', 'DROSHA', 'DUSP4', 'E2F3', 'EED', 'EGFL7', 'EGFR', 'EIF1AX', 'EIF4A2', 'EIF4E', 'ELF3', 'EP300', 'EPAS1', 'EPCAM', 'EPHA3', 'EPHA5', 'EPHA7', 'EPHB1', 'ERBB2', 'ERBB3', 'ERBB4', 'ERCC2', 'ERCC3', 'ERCC4', 'ERCC5', 'ERF', 'ERG', 'ERRFI1', 'ESR1', 'ETV1', 'ETV6', 'EZH1', 'EZH2', 'FAM175A', 'FAM46C', 'FAM58A', 'FANCA', 'FANCC', 'FAT1', 'FBXW7', 'FGF19', 'FGF3', 'FGF4', 'FGFR1', 'FGFR2', 'FGFR3', 'FGFR4', 'FH', 'FLCN', 'FLT1', 'FLT3', 'FLT4', 'FOXA1', 'FOXL2', 'FOXO1', 'FOXP1', 'FUBP1', 'FYN', 'GATA1', 'GATA2', 'GATA3', 'GLI1', 'GNA11', 'GNAQ', 'GNAS', 'GPS2', 'GREM1', 'GRIN2A', 'GSK3B', 'GTF2I', 'H3F3A', 'H3F3B', 'H3F3C', 'HGF', 'HIST1H1C', 'HIST1H2BD', 'HIST1H3A', 'HIST1H3B', 'HIST1H3C', 'HIST1H3D', 'HIST1H3E', 'HIST1H3F', 'HIST1H3G', 'HIST1H3H', 'HIST1H3I', 'HIST1H3J', 'HIST2H3C', 'HIST2H3D', 'HIST3H3', 'HLA-A', 'HLA-B', 'HNF1A', 'HOXB13', 'HRAS', 'ICOSLG', 'ID3', 'IDH1', 'IDH2', 'IFNGR1', 'IGF1', 'IGF1R', 'IGF2', 'IKBKE', 'IKZF1', 'IL10', 'IL7R', 'INHA', 'INHBA', 'INPP4A', 'INPP4B', 'INPPL1', 'INSR', 'IRF4', 'IRS1', 'IRS2', 'JAK1', 'JAK2', 'JAK3', 'JUN', 'KDM5A', 'KDM5C', 'KDM6A', 'KDR', 'KEAP1', 'KIT', 'KLF4', 'KNSTRN', 'KRAS', 'LATS1', 'LATS2', 'LMO1', 'LYN', 'MALT1', 'MAP2K1', 'MAP2K2', 'MAP2K4', 'MAP3K1', 'MAP3K13', 'MAP3K14', 'MAPK1', 'MAPK3', 'MAPKAP1', 'MAX', 'MCL1', 'MDC1', 'MDM2', 'MDM4', 'MED12', 'MEF2B', 'MEN1', 'MET', 'MGA', 'MITF', 'MLH1', 'MLL', 'MLL2', 'MLL3', 'MLL4', 'MPL', 'MRE11A', 'MSH2', 'MSH3', 'MSH6', 'MSI1', 'MSI2', 'MST1', 'MST1R', 'MTOR', 'MUTYH', 'MYC', 'MYCL1', 'MYCN', 'MYD88', 'MYOD1', 'NBN', 'NCOA3', 'NCOR1', 'NEGR1', 'NF1', 'NF2', 'NFE2L2', 'NFKBIA', 'NKX2-1', 'NKX3-1', 'NOTCH1', 'NOTCH2', 'NOTCH3', 'NOTCH4', 'NPM1', 'NRAS', 'NSD1', 'NTHL1', 'NTRK1', 'NTRK2', 'NTRK3', 'NUF2', 'NUP93', 'PAK1', 'PAK7', 'PALB2', 'PARK2', 'PARP1', 'PAX5', 'PBRM1', 'PDCD1', 'PDCD1LG2', 'PDGFRA', 'PDGFRB', 'PDPK1', 'PGR', 'PHOX2B', 'PIK3C2G', 'PIK3C3', 'PIK3CA', 'PIK3CB', 'PIK3CD', 'PIK3CG', 'PIK3R1', 'PIK3R2', 'PIK3R3', 'PIM1', 'PLCG2', 'PLK2', 'PMAIP1', 'PMS1', 'PMS2', 'PNRC1', 'POLD1', 'POLE', 'PPARG', 'PPM1D', 'PPP2R1A', 'PPP4R2', 'PPP6C', 'PRDM1', 'PRDM14', 'PREX2', 'PRKAR1A', 'PRKCI', 'PRKD1', 'PTCH1', 'PTEN', 'PTP4A1', 'PTPN11', 'PTPRD', 'PTPRS', 'PTPRT', 'RAB35', 'RAC1', 'RAC2', 'RAD21', 'RAD50', 'RAD51', 'RAD51B', 'RAD51C', 'RAD51D', 'RAD52', 'RAD54L', 'RAF1', 'RARA', 'RASA1', 'RB1', 'RBM10', 'RECQL', 'RECQL4', 'REL', 'RET', 'RFWD2', 'RHEB', 'RHOA', 'RICTOR', 'RIT1', 'RNF43', 'ROS1', 'RPS6KA4', 'RPS6KB2', 'RPTOR', 'RRAGC', 'RRAS', 'RRAS2', 'RTEL1', 'RUNX1', 'RXRA', 'RYBP', 'SDHA', 'SDHAF2', 'SDHB', 'SDHC', 'SDHD', 'SESN1', 'SESN2', 'SESN3', 'SETD2', 'SETD8', 'SF3B1', 'SH2B3', 'SH2D1A', 'SHOC2', 'SHQ1', 'SLX4', 'SMAD2', 'SMAD3', 'SMAD4', 'SMARCA4', 'SMARCB1', 'SMARCD1', 'SMO', 'SMYD3', 'SOCS1', 'SOS1', 'SOX17', 'SOX2', 'SOX9', 'SPEN', 'SPOP', 'SPRED1', 'SRC', 'SRSF2', 'STAG2', 'STAT3', 'STAT5A', 'STAT5B', 'STK11', 'STK19', 'STK40', 'SUFU', 'SUZ12', 'SYK', 'TAP1', 'TAP2', 'TBX3', 'TCEB1', 'TCF3', 'TCF7L2', 'TEK', 'TERT', 'TET1', 'TET2', 'TGFBR1', 'TGFBR2', 'TMEM127', 'TMPRSS2', 'TNFAIP3', 'TNFRSF14', 'TOP1', 'TP53', 'TP53BP1', 'TP63', 'TRAF2', 'TRAF7', 'TSC1', 'TSC2', 'TSHR', 'U2AF1', 'UPF1', 'VEGFA', 'VHL', 'VTCN1', 'WHSC1', 'WHSC1L1', 'WT1', 'WWTR1', 'XIAP', 'XPO1', 'XRCC2', 'YAP1', 'YES1', 'ZFHX3', 'ZRSR2'])
genesWithNucleosomes = []
for gene in impact468genes:
    info = nucleosomeInfo[nucleosomeInfo['Hugo_Symbol'] == gene]
    if info.shape[0] > 0:
        genesWithNucleosomes.append(gene)

In [28]:
print allImpactMutsUnfiltered[~allImpactMutsUnfiltered['Hugo_Symbol'].isin(genesWithNucleosomes)].shape[0]
print allImpactMutsUnfiltered[allImpactMutsUnfiltered['Hugo_Symbol'].isin(genesWithNucleosomes)].shape[0]

len(genesWithNucleosomes)

NameError: name 'allImpactMutsUnfiltered' is not defined

## DNDS CUMULATIVE FRAC OF MUTATIONS EXPLAINED

In [62]:
def get_score_list(dndsDf, cancerType, hypermutatedStatus):
    cancerTypeDf = dndsDf[dndsDf['cancerType'] == cancerType]
    selectCol = ''
    if hypermutatedStatus == 'hypermutated':
        selectCol = 'qglobal_cv.Hypermutated'
    else:
        selectCol = 'qglobal_cv.Normal'
    l = zip(cancerTypeDf['gene_name'], cancerTypeDf[selectCol])
    return l

def get_ordering(dndsDf, cancerType, hypermutatedStatus):
    cancerTypeDf = dndsDf[dndsDf['cancerType'] == cancerType]
    selectCol = ''
    if hypermutatedStatus == 'hypermutated':
        selectCol = 'qglobal_cv.Hypermutated'
    else:
        selectCol = 'qglobal_cv.Normal'
    l = get_score_list(dndsDf, cancerType, hypermutatedStatus)
    l.sort(key = lambda x: x[1])
    ordering = [i[0] for i in l]
    return ordering

def get_cumulative_mutation_info(maf, ordering, scoreDict, hypermutantStatus, cancerType):
    
    listOfDicts = []
    cntr = 0
    runningSum = 0.0
    totalNMuts = 1.0*maf.shape[0]
    mutCounter = Counter(maf['Hugo_Symbol'])
    for gene in ordering:
        listOfDicts.append({
            'n': cntr, 'frac': runningSum, 'score': scoreDict[gene], 'Hugo_Symbol': gene,
            'hypermutantStatus': hypermutantStatus, 'cancerType': cancerType, 'nmut': mutCounter[gene]
        })
        runningSum += mutCounter[gene]/totalNMuts
        cntr += 1
    return pd.DataFrame(listOfDicts)
        

In [ ]:
#dndsData = pd.read_table(filePathDict['DNDS_RESULTS'])
allImpactMutsUnfiltered

In [43]:
#allImpactMutsFiltered = pd.read_table(filePathDict['IMPACT_BASE_MAF'])
allImpactMutsFiltered['varUuid'] = allImpactMutsFiltered.apply(lambda row: 
        str(row['Start_Position']) + '_' + str(row['Tumor_Seq_Allele2']), axis=1)
allImpactMutsUnfiltered['varUuid'] = allImpactMutsUnfiltered.apply(lambda row: 
        str(row['Start_Position']) + '_' + str(row['Tumor_Seq_Allele2']), axis=1)

oncDict = dict(zip(allImpactMutsFiltered['varUuid'], allImpactMutsFiltered['oncogenic']))
allImpactMutsUnfiltered['oncogenic'] = allImpactMutsUnfiltered['varUuid'].apply(lambda x: 
    oncDict[x] if x in oncDict else None)

In [84]:
#TODO MAKE. A PLOT OUT OF THIS
oncogenicMaf = allImpactMutsUnfiltered[allImpactMutsUnfiltered['oncogenic'].notnull()]
dndsDfs = [
        get_cumulative_mutation_info(oncogenicMaf[(oncogenicMaf['cancerType'] == 'Endometrial Cancer') & (oncogenicMaf['hypermutantStatus'] == 'Hypermutated')], 
            get_ordering(dndsData, 'Endometrial_Cancer', 'hypermutated'),
            dict(get_score_list(dndsData, 'Endometrial_Cancer', 'hypermutated')), 'hypermutated', 'Endometrial_Cancer'),
        get_cumulative_mutation_info(oncogenicMaf[(oncogenicMaf['cancerType'] == 'Endometrial Cancer') & (oncogenicMaf['hypermutantStatus'] == 'Normal')], 
            get_ordering(dndsData, 'Endometrial_Cancer', 'normal'),
            dict(get_score_list(dndsData, 'Endometrial_Cancer', 'normal')), 'normal', 'Endometrial_Cancer'),
    
        get_cumulative_mutation_info(oncogenicMaf[(oncogenicMaf['cancerType'] == 'Colorectal Cancer') & (oncogenicMaf['hypermutantStatus'] == 'Hypermutated')], 
            get_ordering(dndsData, 'Colorectal_Cancer', 'hypermutated'),
            dict(get_score_list(dndsData, 'Colorectal_Cancer', 'hypermutated')), 'hypermutated', 'Colorectal_Cancer'),
        get_cumulative_mutation_info(oncogenicMaf[(oncogenicMaf['cancerType'] == 'Colorectal Cancer') & (oncogenicMaf['hypermutantStatus'] == 'Normal')], 
            get_ordering(dndsData, 'Colorectal_Cancer', 'normal'),
            dict(get_score_list(dndsData, 'Colorectal_Cancer', 'normal')), 'normal', 'Colorectal_Cancer'),      

    
        get_cumulative_mutation_info(oncogenicMaf[(oncogenicMaf['cancerType'] == 'Glioma') & (oncogenicMaf['hypermutantStatus'] == 'Hypermutated')], 
            get_ordering(dndsData, 'Glioma', 'hypermutated'),
            dict(get_score_list(dndsData, 'Glioma', 'hypermutated')), 'hypermutated', 'Glioma'),
        get_cumulative_mutation_info(oncogenicMaf[(oncogenicMaf['cancerType'] == 'Glioma') & (oncogenicMaf['hypermutantStatus'] == 'Normal')], 
            get_ordering(dndsData, 'Glioma', 'normal'),
            dict(get_score_list(dndsData, 'Glioma', 'normal')), 'normal', 'Glioma'),
    
        get_cumulative_mutation_info(oncogenicMaf[(oncogenicMaf['cancerType'] == 'Prostate Cancer') & (oncogenicMaf['hypermutantStatus'] == 'Hypermutated')], 
            get_ordering(dndsData, 'Prostate_Cancer', 'hypermutated'),
            dict(get_score_list(dndsData, 'Prostate_Cancer', 'hypermutated')), 'hypermutated', 'Prostate_Cancer'),
        get_cumulative_mutation_info(oncogenicMaf[(oncogenicMaf['cancerType'] == 'Prostate Cancer') & (oncogenicMaf['hypermutantStatus'] == 'Normal')], 
            get_ordering(dndsData, 'Prostate_Cancer', 'normal'),
            dict(get_score_list(dndsData, 'Prostate_Cancer', 'normal')), 'normal', 'Prostate_Cancer'),
    
        get_cumulative_mutation_info(oncogenicMaf[(oncogenicMaf['cancerType'] == 'Bladder Cancer') & (oncogenicMaf['hypermutantStatus'] == 'Hypermutated')], 
            get_ordering(dndsData, 'Bladder_Cancer', 'hypermutated'),
            dict(get_score_list(dndsData, 'Bladder_Cancer', 'hypermutated')), 'hypermutated', 'Bladder_Cancer'),
        get_cumulative_mutation_info(oncogenicMaf[(oncogenicMaf['cancerType'] == 'Bladder Cancer') & (oncogenicMaf['hypermutantStatus'] == 'Normal')], 
            get_ordering(dndsData, 'Bladder_Cancer', 'normal'),
            dict(get_score_list(dndsData, 'Bladder_Cancer', 'normal')), 'normal', 'Bladder_Cancer')        

          ]

combinedDf = pd.concat(dndsDfs)
combinedDf['cohort'] = combinedDf['cancerType'] + '_' + combinedDf['hypermutantStatus']
combinedDf['scoreAdj'] = combinedDf['score'].apply(lambda x: 1e-10 if x < 1e-10 else x)

In [85]:
#todo properly load the maf etc
combinedDf.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/dndsAndGeneMutations.tsv', index=False, sep='\t')


#print
#allImpactMutsUnfiltered['hypermutantStatus']
